## 文本分类1
### 1. 加载pre_traind model 来进行文本分类
    1.1 准备数据：rotten_tomatoes  初步清洗
    1.2 加载模型：iic/nlp_bert_sentiment-analysis_english-base
    1.3 tqdm 打印进度条  for-loop进行预测
    1.4 比较结果
    
### 2. 文本转向量后分类
        1.1 准备数据：同上
        1.2 加载模型：iic/nlp_corom_sentence-embedding_english-base
        1.3 tqdm 打印进度条 把 text 全部转成 向量
        1.4 用 fit 训练模型， predict 预测结果
        1.5 metrics比较结果


In [1]:
from modelscope import pipeline
from modelscope.utils.constant import Tasks

In [2]:
import pandas as pd 
file_path = "/openbayes/home/rotten_tomatoes_reviews_sample.csv"
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Freshness  10000 non-null  object
 1   Review     10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [3]:
df.columns = ['label','text']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   10000 non-null  object
 1   text    10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [4]:
df.head()

,label,text
0,rotten,The muddled mental states of the characters (...
1,fresh,The Big Wedding is an occasionally charming a...
2,fresh,While 'Captain Phillips' is certainly a thril...
3,rotten,A more mean-spirited and tongue-in-cheek B-mo...
4,fresh,It's easy to poke fun at some of the more far...


In [5]:
mapping = {'rotten':0,'fresh':1}

df['label'] = df['label'].map(mapping)

df.head()

,label,text
0,0,The muddled mental states of the characters (...
1,1,The Big Wedding is an occasionally charming a...
2,1,While 'Captain Phillips' is certainly a thril...
3,0,A more mean-spirited and tongue-in-cheek B-mo...
4,1,It's easy to poke fun at some of the more far...


In [6]:
model_path = "iic/nlp_bert_sentiment-analysis_english-base"

In [7]:
sentiment_task = pipeline(
    task = Tasks.text_classification,
    model = model_path,
    device = "cuda:0"
)

2025-12-31 06:03:36,177 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-12-31 06:03:36,796 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-12-31 06:03:36,880 - modelscope - INFO - initiate model from /output/.cache/modelscope/models/iic/nlp_bert_sentiment-analysis_english-base
2025-12-31 06:03:36,881 - modelscope - INFO - initiate model from location /output/.cache/modelscope/models/iic/nlp_bert_sentiment-analysis_english-base.
2025-12-31 06:03:36,883 - modelscope - INFO - initialize model from /output/.cache/modelscope/models/iic/nlp_bert_sentiment-analysis_english-base


2025-12-31 06:03:37,987 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
/usr/local/lib/python3.10/site-packages/modelscope/utils/checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues r

In [8]:
train = df[:1000]

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1000 non-null   int64 
 1   text    1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [10]:
y_true = train['label']
y_true

0      0
1      1
2      1
3      0
4      1
      ..
995    0
996    1
997    1
998    0
999    0
Name: label, Length: 1000, dtype: int64

In [19]:
from tqdm import tqdm 
y_pred = []

for item in tqdm(train['text']):
    result = sentiment_task(item)
    p_idx = result['labels'].index('Positive')
    p_score = result['scores'][p_idx]
    n_idx = result['labels'].index('Negative')
    n_score = result['scores'][n_idx]
    
    if p_score > n_score:
        y_pred.append(1)
    else:
        y_pred.append(0)

print(y_pred)



100%|██████████| 1000/1000 [00:08<00:00, 124.97it/s]

[0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 

In [21]:
from sklearn.metrics import classification_report

In [23]:
performance = classification_report(
    train['label'], 
    y_pred,
    target_names = ['negative','positive']
)
print(performance)

              precision    recall  f1-score   support

    negative       0.76      0.72      0.74       505
    positive       0.73      0.77      0.75       495

    accuracy                           0.75      1000
   macro avg       0.75      0.75      0.75      1000
weighted avg       0.75      0.75      0.75      1000



In [24]:
model_id = "iic/nlp_corom_sentence-embedding_english-base"

In [26]:
embedding = pipeline(
    task = Tasks.sentence_embedding,
    model = model_id,
    device = "gpu"
)

2025-12-31 06:27:58,460 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0


2025-12-31 06:27:59,248 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
2025-12-31 06:27:59,328 - modelscope - INFO - Got 7 files, start to download ...


Processing 7 items:   0%|          | 0.00/7.00 [00:00<?, ?it/s]

2025-12-31 06:28:20,557 - modelscope - INFO - Download model 'iic/nlp_corom_sentence-embedding_english-base' successfully.
2025-12-31 06:28:20,559 - modelscope - INFO - initiate model from /output/.cache/modelscope/models/iic/nlp_corom_sentence-embedding_english-base
2025-12-31 06:28:20,560 - modelscope - INFO - initiate model from location /output/.cache/modelscope/models/iic/nlp_corom_sentence-embedding_english-base.
2025-12-31 06:28:20,562 - modelscope - INFO - initialize model from /output/.cache/modelscope/models/iic/nlp_corom_sentence-embedding_english-base
2025-12-31 06:28:20,836 - modelscope - WARNING - No preprocessor field found in cfg.
2025-12-31 06:28:20,837 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2025-12-31 06:28:20,837 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/output/.cache/modelscope/models/iic/nlp_corom_sentence-embedding

In [43]:
vector = []

for item in tqdm(train['text']):
    result = embedding(input = {'source_sentence':[item]})
    vector.append(result['text_embedding'][0])


100%|██████████| 1000/1000 [00:07<00:00, 139.52it/s]


In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [50]:
lr_model = LogisticRegression(solver='liblinear', max_iter=1000)

lr_model.fit(vector[:500],train['label'][:500])

LogisticRegression(max_iter=1000, solver='liblinear')

In [51]:
y_pred = lr_model.predict(vector[500:])

In [52]:
print(classification_report(train['label'][500:] , y_pred))

              precision    recall  f1-score   support

           0       0.69      0.66      0.67       259
           1       0.65      0.67      0.66       241

    accuracy                           0.67       500
   macro avg       0.67      0.67      0.67       500
weighted avg       0.67      0.67      0.67       500



In [53]:
accuracy_score(train['label'][500:], y_pred)

0.668